In [133]:
#importing libraries
import pandas as pd
import numpy as np
import scipy.stats as ss
import re

import matplotlib.pyplot as plt
import seaborn as sns

import zipfile
import os

In [134]:
#set your working directory
wd="/Users/gargidangwal/Documents/GitHub/Final_Project"
#import zipfile -- for later maybe
#with zipfile.ZipFile("src/original_data/original_data.zip") as zip_ref:
#    zip_ref.extractall("./original_data") 

In [135]:
#Load in dataset
nlsy_data_raw = pd.read_csv(wd + "/src/original_data/NLSY_79.csv")
var_info_raw = pd.read_excel(wd + "/src/original_data/var_info.xls")

In [136]:
nlsy_data_raw

,R0000100,R0155400,R0169100,R0173600,R0214700,R0214800,R0214900,R0214901,R0215700,R0215710,...,T5024800,T5619500,T5619700,T5619900,T5620100,T5620200,T5620300,T5771500,T5772700,T5772800
0,1,4620,-4,5,3,2,1,1,52,1516,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
1,2,4000,-4,5,3,2,1,1,34,1190,...,52,23000,-4,-4,-4,-4,-4,57,1820,52
2,3,-4,4000,5,3,2,1,1,38,-3,...,52,29000,-4,-4,-4,-4,-4,55,1730,52
3,4,-4,-3,5,3,2,3,2,0,0,...,-5,73000,-4,-4,-4,-4,-4,55,1152,24
4,5,2200,-4,1,3,1,1,1,46,1072,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12681,12682,500,-4,15,3,1,8,4,0,0,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
12682,12683,0,-4,15,3,1,8,4,0,0,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
12683,12684,0,-4,15,3,1,8,4,0,0,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
12684,12685,0,-4,16,2,1,1,1,0,0,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5


In [137]:
var_info_raw

,nlsy_name,questionaire,readable_name,survey_year
0,T5023600,AGEATINT,age,2014
1,T5771500,AGEATINT,age,2016
2,R0216500,AGEATINT,age,1979
3,R0406510,AGEATINT,age,1980
4,R0619010,AGEATINT,age,1981
...,...,...,...,...
124,R6364601,Q13-5_TRUNC_REVISED,earnings,1997
125,R6909701,Q13-5_TRUNC_REVISED,earnings,1999
126,T2076700,Q13-5_TRUNC,earnings,2007
127,T3045300,Q13-5_TRUNC,earnings,2009


In [138]:
#check for duplicates in nlsy_name
if len(var_info_raw) == len(var_info_raw["nlsy_name"].value_counts()):
    print("nlsy_name is unique")
else:
    print("nlsy_name is not unique")

nlsy_name is unique


In [150]:
# Combine readable_name with survey_year


var_info = var_info_raw
var_info = var_info.astype({'survey_year':'str'})
var_info["readable_name_year"] = var_info["readable_name"] + "_" + var_info["survey_year"]

# Change the time-invariant readable variable names

var_info["readable_name_year"] = var_info["readable_name_year"].str.replace("_invariant","")


In [141]:
var_info

,nlsy_name,questionaire,readable_name,survey_year,readable_name_year
0,T5023600,AGEATINT,age,2014,age_2014
1,T5771500,AGEATINT,age,2016,age_2016
2,R0216500,AGEATINT,age,1979,age_1979
3,R0406510,AGEATINT,age,1980,age_1980
4,R0619010,AGEATINT,age,1981,age_1981
...,...,...,...,...,...
124,R6364601,Q13-5_TRUNC_REVISED,earnings,1997,earnings_1997
125,R6909701,Q13-5_TRUNC_REVISED,earnings,1999,earnings_1999
126,T2076700,Q13-5_TRUNC,earnings,2007,earnings_2007
127,T3045300,Q13-5_TRUNC,earnings,2009,earnings_2009


In [142]:
def negative_as_missing(value):
    """Returns pandas missing for negative values
    
    Args:
    value (string/float/integer): any value
    
    Returns:
    (string/float): pandas missing if value is negative; else value itself
    
    """
    
    if type(value) != str:
        if value < 0:
            out = np.nan
        else:
            out = value
    else:
        out = value
    return out

In [143]:
nlsy_df = nlsy_data_raw.applymap(negative_as_missing)
nlsy_df.head()

,R0000100,R0155400,R0169100,R0173600,R0214700,R0214800,R0214900,R0214901,R0215700,R0215710,...,T5024800,T5619500,T5619700,T5619900,T5620100,T5620200,T5620300,T5771500,T5772700,T5772800
0,1,4620.0,NaN,5,3,2,1.0,1.0,52.0,1516.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,4000.0,NaN,5,3,2,1.0,1.0,34.0,1190.0,...,52.0,23000.0,NaN,NaN,NaN,NaN,NaN,57.0,1820.0,52.0
2,3,NaN,4000.0,5,3,2,1.0,1.0,38.0,NaN,...,52.0,29000.0,NaN,NaN,NaN,NaN,NaN,55.0,1730.0,52.0
3,4,NaN,NaN,5,3,2,3.0,2.0,0.0,0.0,...,NaN,73000.0,NaN,NaN,NaN,NaN,NaN,55.0,1152.0,24.0
4,5,2200.0,NaN,1,3,1,1.0,1.0,46.0,1072.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [151]:
### Renaming all variables with readable names
name_dict = dict(zip(var_info["nlsy_name"], var_info["readable_name_year"]))
# name_dict = dict(zip(var_info["nlsy_name"], var_info["readable_name"]))

nlsy_df.rename(columns=(name_dict), inplace=True)

nlsy_df.head()

,individual_id,earnings_1978,sex,employment_status_1979,weeks_worked_1978,hours_worked_1978,age_1979,earnings_1979,employment_status_1980,age_1980,...,hours_worked_2011,weeks_worked_2011,earnings_2013,age_2014,hours_worked_2013,weeks_worked_2013,earnings_2015,age_2016,hours_worked_2015,weeks_worked_2015
0,1,NaN,2,1.0,52.0,1516.0,20,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,2,1.0,34.0,1190.0,20,NaN,1.0,21.0,...,1820.0,52.0,21000.0,55.0,1820.0,52.0,23000.0,57.0,1820.0,52.0
2,3,4000.0,2,1.0,38.0,NaN,17,NaN,1.0,18.0,...,1716.0,52.0,40000.0,53.0,2010.0,52.0,29000.0,55.0,1730.0,52.0
3,4,NaN,2,3.0,0.0,0.0,16,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,73000.0,55.0,1152.0,24.0
4,5,NaN,1,1.0,46.0,1072.0,19,NaN,1.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [152]:
### Keep only those variables that are present in NLSY Variable Information file
droplist = [i for i in nlsy_df.columns if i not in set(var_info["readable_name_year"])]
nlsy_df.drop(droplist,axis=1,inplace=True)

nlsy_df.head()

,individual_id,earnings_1978,sex,employment_status_1979,weeks_worked_1978,hours_worked_1978,age_1979,earnings_1979,employment_status_1980,age_1980,...,hours_worked_2011,weeks_worked_2011,earnings_2013,age_2014,hours_worked_2013,weeks_worked_2013,earnings_2015,age_2016,hours_worked_2015,weeks_worked_2015
0,1,NaN,2,1.0,52.0,1516.0,20,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,2,1.0,34.0,1190.0,20,NaN,1.0,21.0,...,1820.0,52.0,21000.0,55.0,1820.0,52.0,23000.0,57.0,1820.0,52.0
2,3,4000.0,2,1.0,38.0,NaN,17,NaN,1.0,18.0,...,1716.0,52.0,40000.0,53.0,2010.0,52.0,29000.0,55.0,1730.0,52.0
3,4,NaN,2,3.0,0.0,0.0,16,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,73000.0,55.0,1152.0,24.0
4,5,NaN,1,1.0,46.0,1072.0,19,NaN,1.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [153]:
nlsy_df.columns

Index(['individual_id', 'earnings_1978', 'sex', 'employment_status_1979',
       'weeks_worked_1978', 'hours_worked_1978', 'age_1979', 'earnings_1979',
       'employment_status_1980', 'age_1980',
       ...
       'hours_worked_2011', 'weeks_worked_2011', 'earnings_2013', 'age_2014',
       'hours_worked_2013', 'weeks_worked_2013', 'earnings_2015', 'age_2016',
       'hours_worked_2015', 'weeks_worked_2015'],
      dtype='object', length=129)

In [154]:
# Change individual_id into integer (is index variable)

print(nlsy_df["individual_id"].dtype)
nlsy_df["individual_id"] = nlsy_df["individual_id"].astype(int)

nlsy_df.head()


int64


,individual_id,earnings_1978,sex,employment_status_1979,weeks_worked_1978,hours_worked_1978,age_1979,earnings_1979,employment_status_1980,age_1980,...,hours_worked_2011,weeks_worked_2011,earnings_2013,age_2014,hours_worked_2013,weeks_worked_2013,earnings_2015,age_2016,hours_worked_2015,weeks_worked_2015
0,1,NaN,2,1.0,52.0,1516.0,20,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,2,1.0,34.0,1190.0,20,NaN,1.0,21.0,...,1820.0,52.0,21000.0,55.0,1820.0,52.0,23000.0,57.0,1820.0,52.0
2,3,4000.0,2,1.0,38.0,NaN,17,NaN,1.0,18.0,...,1716.0,52.0,40000.0,53.0,2010.0,52.0,29000.0,55.0,1730.0,52.0
3,4,NaN,2,3.0,0.0,0.0,16,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,73000.0,55.0,1152.0,24.0
4,5,NaN,1,1.0,46.0,1072.0,19,NaN,1.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [157]:
### Change from wide to long format

varnames_long = set(list(map(lambda x: re.sub("\_[0-9]{4}$","",x), list(nlsy_df.columns)))[3::])
varnames_long

nlsy_df_long = pd.wide_to_long(df=nlsy_df,stubnames = varnames_long,i=["individual_id"],j="year",sep = "_")

nlsy_df_long.tail()

sex  hours_worked  weeks_worked  age  employment_status  \
individual_id year                                                            
12686         2012    1           NaN           NaN  NaN                NaN   
              2013    1           NaN           NaN  NaN                NaN   
              2014    1           NaN           NaN  NaN                NaN   
              2015    1           NaN           NaN  NaN                NaN   
              2016    1           NaN           NaN  NaN                NaN   

                    earnings  
individual_id year            
12686         2012       NaN  
              2013       NaN  
              2014       NaN  
              2015       NaN  
              2016       NaN